In [ ]:
'''
1. Download FreeSurfer MNI152 1mm nifti file
2. Register parcellation file onto MNI152 1mm file (matching with fsLR) : antsregistration
#https://antspy.readthedocs.io/en/latest/registration.html
(1)ants.registration (fixed = HCP template "MNI152_T1_1mm_brain.nii.gz", moving is the colins27)
(2)using the transformation matrix from ants.registration use that to the ants.apply_transforms(fixed, moving,..) (fixed = HCP global/tempate, the parcellation (von economos))
3. Project volume data onto fsLR space (conte69) : wb_command volume-to-surface : with voxel-enclosing option
'''
#use ANTSPY (https://antspy.readthedocs.io/en/latest/registration.html)
#use HCP: (https://github.com/Washington-University/HCPpipelines), MNI152_T1_1mm_brain.nii.gz in templates folder

import nibabel as nib
import os
import ants
import numpy as np
myatlas_base_path = "/Users/dennis.jungchildmind.org/Downloads/MYATLAS_package_new/"

HCP_template = '/Users/dennis.jungchildmind.org/Downloads/HCPpipelines-master/global/templates/MNI152_T1_2mm_brain.nii.gz'
COLINS27_template = os.path.join(myatlas_base_path, 'maps/Volume/colin27_T1.nii')

#(1) register COLIN27 to HCP template
ants_registration = ants.registration(ants.image_read(HCP_template), ants.image_read(COLINS27_template))
clin27_to_MNI152 = ants.apply_transforms(ants.image_read(HCP_template), ants.image_read(COLINS27_template), ants_registration['fwdtransforms'])
ants.image_write(clin27_to_MNI152, 'clin27_to_MNI152.nii.gz')
#get the transformation matrix
#(2) 



In [ ]:
tmp_t = nib.load(os.path.join(myatlas_base_path,'maps/Surface/HCP_conte69/conte69_32k/gii/cytoarchitecture/vonEK_cell_size_conte69_32k_rh.dscalar.nii'))
print("Header type:", type(tmp_t.header))
print("Header attributes:", dir(tmp_t.header))
print("Shape:", tmp_t.shape)
print("Data type:", tmp_t.get_fdata().dtype)
print("Header info:", tmp_t.header)
print("File path:", tmp_t.get_filename())
# Cifti2Header is not subscriptable, use get_axis method instead
print("Number of axes:", tmp_t.header.number_of_mapped_indices)
print("Axis 0:", tmp_t.header.get_axis(0))
print("Axis 1:", tmp_t.header.get_axis(1))

In [ ]:
ECONOMO_MNI152_VOL = '/Users/dennis.jungchildmind.org/OneDrive - Child Mind Institute/layer_project/cortical_layer/external/economoMNI152volume/economoMNI152volume.nii.gz'
applied_transformation = ants.apply_transforms(ants.image_read(HCP_template), ants.image_read(ECONOMO_MNI152_VOL), ants_registration['fwdtransforms'],
                                               interpolator='multiLabel')

In [ ]:
orig_label = ants.image_read(ECONOMO_MNI152_VOL)
orig_label_data = orig_label.numpy()
orig_label_data[orig_label_data >= 1000] -= 1000 - 1
orig_label_data[orig_label_data >= 1000] -= 1000 -1 - 44 
new_label_data = ants.from_numpy(orig_label_data, spacing=orig_label.spacing, origin=orig_label.origin, direction=orig_label.direction)

In [ ]:
ants.plot(new_label_data, cmap='turbo')

In [ ]:
orig_label_data = applied_transformation.numpy()
orig_label_data[orig_label_data >= 1000] -= 1000 - 1
orig_label_data[orig_label_data >= 1000] -= 1000 -1 - 44 
new_label_data = ants.from_numpy(orig_label_data, spacing=applied_transformation.spacing, origin=applied_transformation.origin, direction=applied_transformation.direction)

In [ ]:
ants.plot(HCP_template, overlay=new_label_data, overlay_cmap='turbo')

In [ ]:
print(applied_transformation)
ants.image_write(applied_transformation, 'applied_transformation.nii.gz')


import neuromaps

d =   neuromaps.transforms.mni152_to_fslr('applied_transformation.nii.gz', fslr_density='32k', method = 'nearest')

In [ ]:

import yaspy
import matplotlib.pyplot as plt
data_base_path = "/Users/dennis.jungchildmind.org/Desktop/exvivo_figures"
base_path = "/Users/dennis.jungchildmind.org/Desktop/exvivo"
subject_name = "I45_new_confidence"
hemisphere = "lh"
layer = "pial"

#surface_file = f"{base_path}/{subject_name}/{hemisphere}.{layer}.32k_fs_LR.surf.gii"
surface_file = '/Users/dennis.jungchildmind.org/Downloads/HCP_S1200_Atlas_Z4_pkXDZ/S1200.L.midthickness_MSMAll.32k_fs_LR.surf.gii'

tmp_dat = nib.load('temp.func.gii')
tmp_dat =d[0].darrays[0].data
#tmp_dat =tmp_dat.darrays[0].data
plotter = yaspy.Plotter(surface_file, hemi=hemisphere)
plotter.overlay(tmp_dat, cmap='jet',vmin=1000,vmax=1044)

#make colormap using the colorvales


# Generate multi-view montage
views = ['lateral', 'medial']
screenshots = [plotter.screenshot(view) for view in views]
montage = yaspy.montage(screenshots, pad=8)
# Create and style figure
fig, ax = plt.subplots(figsize=(6,4))
for spine in ax.spines.values():
    spine.set_visible(False)
plt.imshow(montage)
plt.axis('off')




In [ ]:

volume = "'/Users/dennis.jungchildmind.org/OneDrive - Child Mind Institute/layer_project/cortical_layer/external/economoMNI152volume/applied_transformation.nii'"
surface = "/Users/dennis.jungchildmind.org/Downloads/HCP_S1200_Atlas_Z4_pkXDZ/S1200.L.midthickness_MSMAll.32k_fs_LR.surf.gii"
surface_white = "/Users/dennis.jungchildmind.org/Downloads/HCP_S1200_Atlas_Z4_pkXDZ/S1200.L.white_MSMAll.32k_fs_LR.surf.gii"
surface_pial = "/Users/dennis.jungchildmind.org/Downloads/HCP_S1200_Atlas_Z4_pkXDZ/S1200.L.pial_MSMAll.32k_fs_LR.surf.gii"
roi_file = "/Users/dennis.jungchildmind.org/Downloads/HCPpipelines-master/global/templates/standard_mesh_atlases/L.atlasroi.32k_fs_LR.shape.gii"
metric_out = "temp.func.gii"

# Add the full path to wb_command and use f-string formatting
import os
os.system(f"wb_command -volume-to-surface-mapping {volume} {surface} {metric_out} -ribbon-constrained {surface_white} {surface_pial}")